<a href="https://colab.research.google.com/github/joedockrill/image-scraper/blob/master/ImageScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DuckDuckGo Image Scraper

This was originally an image scraper notebook for creating deep learning datasets.

This has since been turned into an installable library and is much easier to use as you can simply drop a few lines of code into your own notebook as you're experimenting. 

This notebook now shows you how to use the library.

Docs are at [joedockrill.github.io/jmd_imagescraper/](https://joedockrill.github.io/jmd_imagescraper/)

Hugs & kisses, Joe Dockrill. 

## Install



In [1]:
!pip install -q jmd_imagescraper

## Download images

In [ ]:
from pathlib import Path
root = Path().cwd()/"images"

from jmd_imagescraper.core import * # dont't worry, it's designed to work with import *

duckduckgo_search(root, "Cats", "cute kittens", max_results=10)
duckduckgo_search(root, "Dogs", "cute puppies", max_results=10)
duckduckgo_search(root, "Birds", "cute baby ducks and chickens", max_results=10)

# file paths are returned so if you want to snag a list of downloaded files as you go, do this:

# images = []
# images.extend(duckduckgo_search(root, "Cats", "cute kittens", max_results=10))
# images.extend(duckduckgo_search(root, "Dogs", "cute puppies", max_results=10))
# images.extend(duckduckgo_search(root, "Birds", "cute baby ducks and chickens", max_results=10))
# images

## Changing params across multiple searches

In [ ]:
# If you're going to override default params across multiple searches you can use a 
# dictionary like this (so you can change search params for the entire dataset once).

params = {
    "max_results": 10,             # this can go up to 477 at the time of writing
    "img_size":    ImgSize.Cached, 
    "img_type":    ImgType.Photo,
    "img_layout":  ImgLayout.Square,
    "img_color":   ImgColor.Purple
}

duckduckgo_search(root, "Nice", "nice clowns", **params)
duckduckgo_search(root, "Scary", "scary clowns", **params)

## Deleting all images

In [5]:
rmtree(root)

## Displaying the image cleaner

Use this to get rid of unsuitable images without leaving your notebook

In [ ]:
from jmd_imagescraper.imagecleaner import *

display_image_cleaner(root)

## Create a zip to download or transfer to google drive

In [11]:
# create zip

ZIP_NAME = "images.zip" # maybe change this?

!rm -f {ZIP_NAME}
!zip -q -r {ZIP_NAME} {root}

In [ ]:
# download to your local system

from google.colab import files
files.download(ZIP_NAME)

In [ ]:
# copy to google drive 

from google.colab import drive
import shutil

DESTINATION_FOLDER = "Datasets" # where would you like this in Google Drive?

drive.mount("/content/drive") 
folder = Path("/content/drive/My Drive")/DESTINATION_FOLDER
folder.mkdir(parents=True, exist_ok=True)

shutil.copyfile(ZIP_NAME, str(folder/ZIP_NAME))

## Create a CSV file of URLs

If you'd rather distribute a file with the image URLs and labels and have people download the images themselves you can do so here.

In [15]:
CSV_NAME = "images.csv" # maybe change this?

!rm -f {CSV_NAME}

csv = Path.cwd()/CSV_NAME
save_urls_to_csv(csv, "Nice", "nice clowns", max_results=5)
save_urls_to_csv(csv, "Scary", "scary clowns", max_results=5)